# Hierarchical Quantum Classifier

Code for paper "Hierarchical Quantum Classifier" (2018 Grant et al.). Investigate Tree Tensor Network (TTN) and Multi-scale Entanglement Renormalization Ansatz (MERA) structures. 

In [1]:
import pennylane as qml
from pennylane import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import compat

### 1. Data Preparation

MNIST data loading and processing procedures.

In [58]:
classes = [0, 1]
num_classes = len(classes)
input_shape = (28, 28, 1)

In [88]:
def load_and_process_data():
    (X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
    
    X_train, X_test = X_train.astype("float32") / 255, X_test.astype("float32") / 255
    X_train, X_test = np.expand_dims(X_train, -1), np.expand_dims(X_test, -1)

    return X_train, X_test, Y_train, Y_test

In [91]:
def classes_filter(X, Y, classes = classes):
    
    filters = []
    for i in classes:
        filter_i = np.where(Y == i)
        filters.append(filter_i)
    filters = np.array(filters)
    
    X_filtered = np.array([X[f] for f in filters])
    Y_filtered = np.array([Y[f] for f in filters])
    return X_filtered, Y_filtered

In [128]:
X_train, X_test, Y_train, Y_test = load_and_process_data()
print(Y_train.shape)
filters = []
X_filtered = []

filter_i = np.where( (Y_train ==1 ))
print(filter_i)



(60000,)
(array([    3,     6,     8, ..., 59979, 59984, 59994]),)


We need to reduce the size of input data for quantum embedding. Here we introduce several options including CNN, PCA, Resizing.

In [65]:
def CNN(X, Y, batch_size = 128, epochs = 10, num_datasets = 100):
    model = keras.Sequential(
    [
        keras.Input(shape = input_shape),
        layers.Conv2D(32, kernel_size = (3,3), activation = "relu"),
        layers.MaxPooling2D(pool_size = (2,2)),
        layers.Conv2D(64, kernel_size = (3,3), activation = "relu"),
        layers.MaxPooling2D(pool_size = (2,2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation = "softmax")
    ])
    
    model.summary()
    
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics =["accuracy"])
    
    X_train, Y_train = classes_filter(X, Y, classes = classes)
    X_train, Y_train = 
    model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_split = 0.1)
    
    extractor = keras.Model(inputs=model.inputs,
                            outputs=[layer.output for layer in model.layers])
    
    features = []
    for X_i in X_train:
        features_i = extractor(X_i)
        features.append(features_i[6])
        
    tf.compat.v1.enable_eager_execution()
    
    return features

In [66]:
X_train, X_test, Y_train, Y_test = load_and_process_data()
X_train, Y_train = 

In [67]:
features = CNN(X_train, Y_train)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                

ValueError: in user code:

    /Users/tak/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/tak/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/tak/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/tak/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/tak/anaconda3/envs/QC/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/tak/anaconda3/envs/QC/lib/p

### 2. Hierarchical Quantum Circuits

In [68]:
def qubit_encoding(vector):
    for i in range(num_wires):
        qml.RY(vector[i] * 2, wires = i)


In [69]:
def U_block_t(params, wires):
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[0], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_block_b(params, wires):
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[0], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])

In [71]:
num_wires = 8
dev = qml.device('default.qubit', wires = num_wires)

@qml.qnode(dev)
def variational_circuit(params, features):
    
    qubit_encoding(features)
    
    #MERA circuit
    #D blocks on (1,2), (3,4), (5,6), (7,8)    
    U_block_t(params[:2],wires=[1,2])
    U_block_b(params[2:4],wires=[3,4])
    U_block_t(params[4:6],wires=[5,6])
    #U_block_t(params[6:8],wires=[7,8])

    #U blocks on (0,1), (2,3), (4,5), (6,7), (8,9)
    U_block_t(params[6:8],wires=[0,1])
    U_block_d(params[8:10],wires=[2,3])
    U_block_t(params[10:12],wires=[4,5])
    U_block_d(params[12:14],wires=[6,7])
    #U_block_d(params[:2],wires=[1,2])
    
    #D in the middle
    U_block_t(params[14:16],wires=[2,5])
    
    #Two U's
    U_block_t(params[16:18],wires=[1,2])
    U_block_d(params[18:20],wires=[5,6])
    
    #Last U
    U_block_t(params[20:22],wires=[2,5])
    
    return qml.expval(qml.PauliZ(5))

In [73]:
def square_loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    return loss / len(labels)

def cost(params, X, Y):
    predictions = [variational_circuit(params, x) for x in X]
    return square_loss(Y, predictions)

In [75]:
steps = 50
batch_size = 20
num_params = 22

def circuit_training():
    opt = qml.NesterovMomentumOptimizer(0.5)
    params = np.random.randn(num_params)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda p: cost(p, X_batch, Y_batch), params)
        if it % 10 == 0:
            print("Steps: ", it, " Cost: ", cost_new)
    
    return params